In [ ]:
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
from IPython.display import clear_output

ko_list_df = pd.read_csv("ko_list.txt", sep="\t", header=None, index_col=0)
ko_list_df.columns = ["gene_name"]
ko_list_df.index.name = "KO"
ko_list = (ko_list_df.index + ": " + ko_list_df["gene_name"]).to_dict()

ko_formatted_names = (ko_list_df.index + ": " + ko_list_df["gene_name"]).tolist()
ko_formatted_names_w = widgets.Select(options=ko_formatted_names, description="KO list", rows=40, layout=widgets.Layout(width='700px'))

with open("genomes.txt", "r") as f:
    genomes = f.read()
genomes = genomes.split()

genomes_w = widgets.SelectMultiple(
    options=genomes,
    value=genomes,
    rows=40,
    description='Genomes',
    disabled=False
)

init_ko_list = ["K03821", "K00370", "K00371", "K00374", "K00373"]
ko_input = widgets.TagsInput(
    value=init_ko_list,
    description="Give some KEGG KOs",
    allow_duplicates=False
)

filter_text = widgets.Text(
    value="",
    description="Filter KO list",
    placeholder="Type to filter..."
)

# Create an interactive output for the heatmap
heatmap_output = widgets.Output()

# Function to update the heatmap based on widget values
def update_heatmap(change):
    with heatmap_output:
        # Clear previous output
        clear_output(wait=True)

        # Get the selected genomes and KOs
        selected_genomes = genomes_w.value
        selected_kos = ko_input.value

        # Generate the matrix based on selected genomes and KOs
        mk = matrix_ko(selected_genomes, selected_kos)

        # Plot the heatmap
        fig = px.imshow(mk, aspect="auto")
        fig.update_layout(height=800)
        fig.update_xaxes(tickangle=-45,
                         showgrid=True,
                         gridwidth=1,
                         gridcolor='blue')
        fig.show(config={"editable": True})

# Register the update function with the widgets
genomes_w.observe(update_heatmap, names='value')
ko_input.observe(update_heatmap, names='value')

# Function to filter KO list based on text input
def filter_ko_list(change):
    # Get the filter text
    filter_value = filter_text.value.lower()

    # Filter KO list based on the text input
    filtered_ko_list = [ko for ko in ko_formatted_names if filter_value in ko.lower()]

    # Update the KO formatted names widget
    ko_formatted_names_w.options = filtered_ko_list

# Register the filter function with the text widget
filter_text.observe(filter_ko_list, names='value')

# Display the initial heatmap
update_heatmap(None)

# Display the widgets and the interactive output
widgets.HBox([
    widgets.VBox([genomes_w, ko_input], layout=widgets.Layout(max_width='400px')),
    heatmap_output,
    widgets.VBox([filter_text, ko_formatted_names_w])
])
